In [8]:
import os
import io
import sys
import time
import numpy as np
import pandas as pd
import requests as r
from requests.exceptions import ConnectionError, Timeout
import mysql.connector
from pprint import pprint
from datetime import datetime, date, timedelta
from dotenv import dotenv_values
from IPython.display import clear_output
from mysql.connector.errors import Error as MySQK_Error

pd.set_option('display.max_rows', 35)

In [9]:
ROOT_PATH = os.getcwd()
SERVICE_ID_FILE = os.path.join(ROOT_PATH, 'id_services.csv')
ENV_PATH = os.path.join(ROOT_PATH, '.env')
MAIN_SQL_QUERY_FILE_PATH = os.path.join(ROOT_PATH, 'get_data_query.sql')
OTHER_SQL_QUERY_FILE_PATH = os.path.join(ROOT_PATH, 'available_visits_sql_query.sql')

DF_FILE_NAME = 'dataset.csv'

if os.path.exists(ENV_PATH):
    ENV_VAR = dotenv_values(ENV_PATH)

df = pd.DataFrame()
dataset_path = os.path.join(ROOT_PATH, DF_FILE_NAME)
if os.path.exists(dataset_path):
    df = pd.read_csv(dataset_path, sep=';')

## Классы пользовательских исключений

In [10]:
class DataError(Exception):
    def __init__(self, *args):
        if args:
            self.message =  ' '.join(args)
        else:
            self.message = 'Data is empty.\n'
    def __str__(self):
        return self.message


## Общие вспомогательные ф-ии

In [11]:
def to_df(data, columns=None):
    """ преобразуем список кортежей в dataframe """
    return pd.DataFrame(np.array(data), columns=columns)

def read_json_file(path):
    """ Чтение json файла """
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data


## Вспомогательные ф-ии для сбора внутренних данных

In [12]:
def get_connection():
    """ подключаемся к MySQL базе """
    conn = mysql.connector.connect(host=ENV_VAR['DB_HOST'],
                                   port=ENV_VAR['DB_PORT'],
                                   database=ENV_VAR['DB_NAME'],
                                   user=ENV_VAR['DB_USER'],
                                   password=ENV_VAR['DB_PASS'])
    if conn.is_connected():
        print('Connected to MySQL database')
        return conn
    else:
        raise MySQK_Error

def get_data(conn, sql_query, *args):
    """ получаем данные из базы """
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql_query, args)
            return cursor.fetchall()
    except MySQK_Error as e:
        raise e


In [13]:
raise KeyboardInterrupt

KeyboardInterrupt: 

# Внутренние данные

In [ ]:
sid_df = pd.read_csv(SERVICE_ID_FILE, sep=';')
columns = ['id', 'service_id', 'cost', 'name', 'item_code', 'medical_specialty', 
           'amount_doctors', 'service_date', 'sales']


In [ ]:
df = pd.DataFrame(columns=columns)

try:
    mysql_conn = get_connection()
except MySQK_Error as e:
    print(e)
    raise KeyboardInterrupt

sql_query = None
with open(MAIN_SQL_QUERY_FILE_PATH, 'r') as f:
    sql_query = f.read()

if not sql_query:
    print('sql файл не прочитан')
    raise KeyboardInterrupt
    
for sid in sid_df['sid']:   
    data = None
    for _ in range(3):
        try:
            data = get_data(mysql_conn, sql_query, sid)
        except MySQK_Error as e:
            print(e)
            mysql_conn.close()
            time.sleep(10)
            mysql_conn = get_connection()
            continue
        else:
            break
    if not data:
        raise DataError

    temp_df = to_df(data, columns)
    temp_df.loc[:, 'service_type'] = [sid_df.loc[sid_df['sid'] == sid]['type'].values[0] 
                                      for _ in range(temp_df.shape[0])]
    df = pd.concat([df, temp_df])
    clear_output()
    display('df: ', df)

if mysql_conn.is_connected():
    mysql_conn.close()

df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
raise KeyboardInterrupt

# Уникальные услуги

In [ ]:
f_path = os.path.join(ROOT_PATH, 'uniq_services.csv')
unique_services = pd.read_csv(f_path, sep=';')

In [ ]:
def set_uniq_serv_feat(row):
    row['unique_service'] = unique_services.loc[
        unique_services['id'] == row['service_id']]['unique_service'].values[0]
    return row


In [ ]:
df['unique_service'] = None
df = df.apply(set_uniq_serv_feat, axis=1)
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
raise KeyboardInterrupt

# Количество доступных приёмов в день

In [ ]:
def set_avail_recept_feat(row):
    global mysql_conn
    clear_output()
    f_date = datetime.strptime(row['service_date'], '%d.%m.%Y').strftime('%Y-%m-%d')
    sid = row['service_id']
    data = None
    
    if not sql_query:
        print('sql файл не прочитан')
        raise KeyboardInterrupt
    
    for _ in range(3):
        try:
            data = get_data(mysql_conn, sql_query, f_date, sid, f_date)
        except MySQK_Error as e:
            print(e)
            if mysql_conn.is_connected():
                mysql_conn.close()
            time.sleep(20)
            mysql_conn = get_connection()
            continue
        else:
            break

    if not data:
        raise DataError
    
    row['number_of_receptions'] = data[0][0]
    display(row)
    return row

In [ ]:
df['number_of_receptions'] = None

try:
    mysql_conn = get_connection()
except MySQK_Error as e:
    print(e)
    raise KeyboardInterrupt

with open(OTHER_SQL_QUERY_FILE_PATH, 'r') as f:
    sql_query = f.read()
    
df = df.apply(set_avail_recept_feat, axis=1)

if mysql_conn.is_connected():
        mysql_conn.close()

df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
df


In [ ]:
raise KeyboardInterrupt

# Внешние данные

# Не используется

## Курс доллара за указанную дату

#### Информация о сервисе http://www.cbr.ru/development/SXML/

Получить курсы для валют:
url = http://www.cbr.ru/scripts/XML_daily.asp?date_req=02/03/2002

In [ ]:
# import xml.etree.ElementTree as ET

# def set_usd_cours_feat(row):
#     row['USD_rate'] = usd_cours[row['service_date']]
#     return row    
    

# def get_usd_cours(list_dates):
#     try:
#         courses = dict()
#         for c_date in list_dates:
#             clear_output()
            
#             f_date = (datetime.strptime(c_date, '%d.%m.%Y') - timedelta(days=1)).strftime('%d/%m/%Y')
#             link = f'http://www.cbr.ru/scripts/XML_daily.asp?date_req={f_date}'

#             for _ in range(5):
#                 try:
#                     response = r.get(link)
#                     if response.status_code not in [200]:
#                         time.sleep(15)
#                         continue
#                     print('Успешно: ', f_date, response.status_code)
#                     break
#                 except ConnectionError as e:
#                     print('Ошибка соединения:\n', e)
#                     time.sleep(30)
#                     continue
#                 except Timeout as e:
#                     print('Ошибка соединения:\n', e)
#                     time.sleep(120)
#                     continue
#             else:
#                 print(f'Сервер не отвечал 5 раз подряд: {response.status_code}.\n'
#                       f'Операция прервана на "{list(courses.keys())[-1]}".')
#                 return courses

#             root = ET.fromstring(response.content)
#             for elem in root:
#                 if elem[1].text == 'USD':
#                     courses[c_date] = float(elem[4].text.replace(',', '.'))
#                     print(elem[1].text, elem[4].text)
#                     break
#             print(list(courses.keys())[-1], courses[list(courses.keys())[-1]])
#             time.sleep(5)
#         clear_output()
#         print(f'Данные получены полностью.')
#         return courses
#     except KeyboardInterrupt:
#         if len(list(courses.keys())) != 0:
#             print(f'Операция прервана на "{list(courses.keys())[-1]}".')
#         return courses
#     except Exception as e:
#         print('Неизвестная ошибка:\n', e)
#         if len(list(courses.keys())) != 0:
#             print(f'Операция прервана на "{list(courses.keys())[-1]}".')
#         return courses


In [ ]:
# usd_cours = get_usd_cours(df['service_date'].unique())
# usd_cours

In [ ]:
# df['USD_rate'] = None
# df = df.apply(set_usd_cours_feat, axis=1)
# df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
# df

In [ ]:
# import json
# file_name = 'usd_courses.json'
# if os.path.exists(os.path.join(ROOT_PATH, file_name)):
#     mode = 'a'
# else:
#     mode = 'w'

# with open(os.path.join(ROOT_PATH, file_name), mode) as f:
#     json.dump(usd_cours, f)

In [ ]:
# raise KeyboardInterrupt

# Holidays

In [ ]:
list_holidays = ['22.02.2017', '23.02.2017', '24.02.2017', '07.03.2017', '08.03.2017', '01.05.2017', 
'08.05.2017', '09.05.2017', '12.06.2017', '03.11.2017', '04.11.2017', '06.11.2017', '01.01.2018', 
'02.01.2018', '03.01.2018', '04.01.2018', '05.01.2018', '06.01.2018', '07.01.2018', '08.01.2018', 
'22.02.2018', '23.02.2018', '07.03.2018', '08.03.2018', '09.03.2018', '28.04.2018', '30.04.2018', 
'01.05.2018', '02.05.2018', '08.05.2018', '09.05.2018', '09.06.2018', '11.06.2018', '12.06.2018', 
'04.11.2018', '05.11.2018', '29.12.2018', '31.12.2018', '01.01.2019', '02.01.2019', '03.01.2019', 
'04.01.2019', '05.01.2019', '06.01.2019', '07.01.2019', '08.01.2019', '22.02.2019', '23.02.2019', 
'07.03.2019', '08.03.2019', '30.04.2019', '01.05.2019', '02.05.2019', '03.05.2019', '08.05.2019', 
'09.05.2019', '10.05.2019', '11.06.2019', '04.11.2019', '31.12.2019', '01.01.2020', '02.01.2020', 
'03.01.2020', '04.01.2020', '05.01.2020', '06.01.2020', '07.01.2020', '08.01.2020', '23.02.2020', 
'24.02.2020', '08.03.2020', '09.03.2020', '30.03.2020', '31.03.2020', '01.04.2020', '02.04.2020', 
'03.04.2020', '04.04.2020', '05.04.2020', '06.04.2020', '07.04.2020', '08.04.2020', '09.04.2020', 
'10.04.2020', '11.04.2020', '12.04.2020', '13.04.2020', '14.04.2020', '15.04.2020', '16.04.2020', 
'17.04.2020', '18.04.2020', '19.04.2020', '20.04.2020', '21.04.2020', '22.04.2020', '23.04.2020', 
'24.04.2020', '25.04.2020', '26.04.2020', '27.04.2020', '28.04.2020', '29.04.2020', '30.04.2020', 
'01.05.2020', '02.05.2020', '03.05.2020', '04.05.2020', '05.05.2020', '06.05.2020', '07.05.2020', 
'08.05.2020', '09.05.2020', '10.05.2020', '11.05.2020', '11.06.2020', '12.06.2020', '24.06.2020', 
'01.07.2020', '03.11.2020', '04.11.2020', '31.12.2020', '01.01.2021', '02.01.2021', '03.01.2021', 
'04.01.2021', '05.01.2021', '06.01.2021', '07.01.2021', '08.01.2021', '22.02.2021', '23.02.2021', 
'08.03.2021', '30.04.2021', '01.05.2021', '03.05.2021', '04.05.2021', '05.05.2021', '06.05.2021', 
'07.05.2021', '09.05.2021', '10.05.2021', '11.05.2021', '12.05.2021', '14.05.2021', '01.11.2021', 
'02.11.2021', '03.11.2021', '04.11.2021', '05.11.2021', '31.12.2021', '01.01.2022', '02.01.2022', 
'03.01.2022', '04.01.2022', '05.01.2022', '06.01.2022', '07.01.2022', '22.02.2022', '23.02.2022', 
'05.03.2022', '07.03.2022', '08.03.2022', '01.05.2022', '02.05.2022', '03.05.2022', '09.05.2022', 
'10.05.2022', '12.06.2022', '13.06.2022']


In [ ]:
def set_holiday_feat(row):
    if row['service_date'] in list_holidays:
        row['holiday'] = 1
    else:
        row['holiday'] = 0
    return row

In [ ]:
df['holiday'] = None
df = df.apply(set_holiday_feat, axis=1)


In [ ]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
raise KeyboardInterrupt

# Weekend days

In [ ]:
def set_weekend_feat(row):
    weekend_days = [5, 6]
    curr_date = datetime.strptime(row['service_date'], '%d.%m.%Y')
    
    if date(curr_date.year, curr_date.month, curr_date.day).weekday() in weekend_days:
        row['weekend'] = 1
    else:
        row['weekend'] = 0
    return row


In [ ]:
df['weekend'] = None
df = df.apply(set_weekend_feat, axis=1)


In [ ]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
df

In [ ]:
raise KeyboardInterrupt

# Week days

In [ ]:
def set_weekday_feat(row):
    """
        Порядковый номер дня недели  начиная с 0 - понедельник.
    """
    wds = {0:'Mon',
           1:'Tue',
           2:'Wed',
           3:'Thu',
           4:'Fri',
           5:'Sat',
           6:'Sun'}
    cd = datetime.strptime(row['service_date'], '%d.%m.%Y')
    wd = date(cd.year, cd.month, cd.day).weekday()
    row['weekday'] = wds[wd]
    return row


In [ ]:
df['weekday'] = None
df = df.apply(set_weekday_feat, axis=1)


In [ ]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
df

In [ ]:
raise KeyboardInterrupt

# Season

In [ ]:
def set_season_feat(row):
    
    ss = {'summer': [6, 7, 8],
          'autumn': [9, 10, 11], 
          'winter': [1, 2, 12],
          'spring': [3, 4, 5]
         }
    
    curr_date = datetime.strptime(row['service_date'], '%d.%m.%Y')
    if curr_date.month in ss['summer']:
        row['season'] = 'summer'
    elif curr_date.month in ss['autumn']:
        row['season'] = 'autumn'
    elif curr_date.month in ss['winter']:
        row['season'] = 'winter'
    elif curr_date.month in ss['spring']:
        row['season'] = 'spring'
    return row

In [ ]:
df['season'] = None
df = df.apply(set_season_feat, axis=1)


In [ ]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
df

In [ ]:
raise KeyboardInterrupt

# Pandemic COVID-19

In [ ]:
def set_pandemic_feat(row):
    if row['service_date'] in pandemic_dates:
        row['pandemic'] = 1
    else:
        row['pandemic'] = 0
    return row


In [ ]:
start_date = datetime(2020, 3, 11)
end_date = datetime(2022, 7, 1)

pandemic_dates = pd.date_range(start_date, end_date).strftime('%d.%m.%Y').tolist()

In [ ]:
pandemic_dates

In [ ]:
df['pandemic'] = None
df = df.apply(set_pandemic_feat, axis=1)


In [ ]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
df

Примерно половина всех приёмов, оказанных за период с 26.01.2017 по 05.08.2022 приходится на период пандемии.

In [ ]:
raise KeyboardInterrupt

# Month

In [ ]:
def set_month_feat(row):
    month_names = {
        1: 'Jan',
        2: 'Feb',
        3: 'Mar',
        4: 'Apr',
        5: 'May',
        6: 'Jun',
        7: 'Jul',
        8: 'Aug',
        9: 'Sep',
        10: 'Oct',
        11: 'Nov',
        12: 'Dec'
    }
    curr_month = datetime.strptime(row['service_date'], '%d.%m.%Y').month
    row['month'] = month_names[curr_month]
    return row


In [ ]:
df['month'] = None
df = df.apply(set_month_feat, axis=1)


In [16]:
df.drop('profit', axis=1, inplace=True)

In [29]:
df.to_csv(os.path.join(ROOT_PATH, DF_FILE_NAME), encoding='utf-8', index=False, sep=';')

In [ ]:
raise KeyboardInterrupt

In [32]:
df

,service_id,cost,name,item_code,medical_specialty,amount_doctors,service_date,sales,service_type,weekend,weekday,holiday,season,unique_service,pandemic,month
0,3100,600,"Прием (осмотр, консультация) врача-акушера-гин...",В01.001.001,акушерство и гинекология,1,26.01.2017,7,Приёмы,0,Thu,0,winter,0,0,Jan
1,3100,600,"Прием (осмотр, консультация) врача-акушера-гин...",В01.001.001,акушерство и гинекология,1,27.01.2017,1,Приёмы,0,Fri,0,winter,0,0,Jan
2,3100,600,"Прием (осмотр, консультация) врача-акушера-гин...",В01.001.001,акушерство и гинекология,2,28.01.2017,8,Приёмы,1,Sat,0,winter,0,0,Jan
3,3100,600,"Прием (осмотр, консультация) врача-акушера-гин...",В01.001.001,акушерство и гинекология,1,29.01.2017,4,Приёмы,1,Sun,0,winter,0,0,Jan
4,3100,600,"Прием (осмотр, консультация) врача-акушера-гин...",В01.001.001,акушерство и гинекология,3,30.01.2017,4,Приёмы,0,Mon,0,winter,0,0,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143818,120853,5000,Видеоколоноскопия,A03.18.001,эндоскопия,1,13.07.2022,1,Функциональная Диагностика,0,Wed,0,summer,0,0,Jul
143819,120853,5000,Видеоколоноскопия,A03.18.001,эндоскопия,1,16.07.2022,2,Функциональная Диагностика,1,Sat,0,summer,0,0,Jul
143820,120853,5000,Видеоколоноскопия,A03.18.001,эндоскопия,1,21.07.2022,1,Функциональная Диагностика,0,Thu,0,summer,0,0,Jul
143821,120853,5000,Видеоколоноскопия,A03.18.001,эндоскопия,1,29.07.2022,1,Функциональная Диагностика,0,Fri,0,summer,0,0,Jul
